In [35]:
from utils.processor import *
from utils.solver import *
from config import *
from utils.tom_api import *

In [36]:
data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')
origin = (HOME['lat'], HOME['lon'])
# dest = (PACIFIC_CENTER['lat'], PACIFIC_CENTER['lon'])
ttapi = TomSearchAPI(API_KEY)
dest = ttapi.find_location('Five Guys Vancouver')
print(dest)
dest = dest['location']
dest = (dest['lat'], dest['lon'])
dest

{'poi_name': 'Five Guys', 'address': '635 Robson Street, Vancouver BC V6B 1H3', 'location': {'lat': 49.28106, 'lon': -123.11902}}


(49.28106, -123.11902)

In [37]:
topK = 10
stay_duration = 1.5 * 60 
parking_budget = 5.00

processor = Processor(datapath, origin, dest, topK, stay_duration, parking_budget)
variables = processor.form_variables_dict()
variables

(7954, 21)
(618, 23)
IDX - 0
Time limit: 2 	 Rate: 3.0
*** Travel time: 16.65 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 1
Time limit: 2 	 Rate: 3.0
*** Travel time: 16.55 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 2
Time limit: 2 	 Rate: 4.0
Skipping due to fee constraint
IDX - 3
Time limit: 2 	 Rate: 3.0
*** Travel time: 16.483333333333334 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 4
Time limit: 2 	 Rate: 3.0
*** Travel time: 16.45 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 5
Time limit: 2 	 Rate: 5.0
Skipping due to fee constraint
IDX - 6
Time limit: 2 	 Rate: 5.0
Skipping due to fee constraint
IDX - 7
Time limit: 2 	 Rate: 1.0
*** Travel time: 16.25 ***
out: 1 - in: 0
~~~ Parking Status: 2 ~~~
{'b_1': 1.0, 'constraint_coeffs': {'parking_fee': 1.5, 'savings': 3.5, 'expected_avail': 2, 'et_next_out': 10.0, 'et_

{'y_1': {'b_1': 1.0,
  'constraint_coeffs': {'parking_fee': 1.5,
   'savings': 3.5,
   'expected_avail': 2,
   'et_next_out': 10.0,
   'et_next_in': 21.0,
   'walk_time': 3.083,
   'travel_et': 16.25},
  'meterid': '600903',
  'latln': (49.27954585600124, -123.12074549294861)},
 'y_2': {'b_2': 1.0,
  'constraint_coeffs': {'parking_fee': 1.5,
   'savings': 3.5,
   'expected_avail': 2,
   'et_next_out': 15.0,
   'et_next_in': 26.0,
   'walk_time': 3.75,
   'travel_et': 16.083333333333332},
  'meterid': '600921',
  'latln': (49.27919165701666, -123.12128250104442)}}

In [38]:
solver = Solver(variables)
solver.solve_LP()
soln = solver.get_soln_summary()
soln

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/8ed6846585b44d26a94fb0ce646006c7-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/8ed6846585b44d26a94fb0ce646006c7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 23 RHS
At line 29 BOUNDS
At line 32 ENDATA
Problem MODEL has 5 rows, 2 columns and 10 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 2 (0) columns and 10 (0) elements
0  Obj 0 Primal inf 1.9999998 (2)
2  Obj 1
Optimal - objective value 1
Optimal objective 1 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.

,var,optimal_weight,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
1,y_2,0.5,1.5,2,16.083333,3.750,15.0,26.0
0,y_1,0.5,1.5,2,16.250000,3.083,10.0,21.0


# Visualization

In [39]:
import folium
fmap = folium.Map(location=dest, zoom_start=16, width="100%", height="50%")

# folium.Marker(
#     location=origin, icon=folium.Icon(color="red",icon="home", prefix='fa')
# ).add_to(fmap)

folium.Marker(
    location=dest, icon=folium.Icon(color="red",icon="location-dot", prefix='fa')
).add_to(fmap)


for i, spot in enumerate(soln.loc[soln.optimal_weight > 0]['var']):
    print(spot)
    rank = i+1
    spot_loc = variables[spot]['latln']
    folium.Marker(
        location = spot_loc,
        popup = f'Rank-{rank}',
        icon = folium.Icon(color="blue", icon="circle-parking", prefix='fa')
    ).add_to(fmap)
    
# for r_id, result in enumerate(res['results']):
#     folium.Marker(
#         location = [result['position']['lat'], result['position']['lon']],
# #         popup = result['poi']['name'],
#         popup = generate_popup(r_id, result),
#         icon = get_icon(poi_category)
#     ).add_to(fmap)

fmap

y_2
y_1
